<a href="https://colab.research.google.com/github/keerthucit/EIP2Assignment/blob/master/AJindul_Assignment4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# https://keras.io/
!pip install -q keras
import keras

In [0]:
import keras
import math

from keras.datasets import cifar10
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, Input,GlobalAveragePooling2D, AveragePooling2D, merge, Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization, SeparableConv2D
from keras.layers import Concatenate
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import *

In [0]:
# this part will prevent tensorflow to allocate all the avaliable GPU Memory
# backend
import tensorflow as tf
from keras import backend as k

# Don't pre-allocate memory; allocate as-needed
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

# Create a session with the above options specified.
k.tensorflow_backend.set_session(tf.Session(config=config))

In [0]:
# Hyperparameters
batch_size = 64
num_classes = 10
epochs = 150
l = 8
num_filter = 32
compression = 0.4
dropout_rate = 0.2

In [0]:
# Load CIFAR10 Data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]

# convert to one hot encoing 
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [0]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

In [0]:
imgAug = ImageDataGenerator(rotation_range=60,width_shift_range=0.1, zoom_range=0.1,horizontal_flip=True, height_shift_range=0.1)
imgAug.fit(x_train)

In [0]:
# Dense Block
def add_denseblock(input, num_filter = 32, dropout_rate = 0.2):
    global compression
    temp = input
    for _ in range(l):
        BatchNorm = BatchNormalization()(temp)
        relu = Activation('relu')(BatchNorm)
        Conv2D_1_1 = Conv2D(int(num_filter*compression*5), (1,1), use_bias=False ,padding='same')(relu)
        
        BatchNorm1 = BatchNormalization()(Conv2D_1_1)
        relu1 = Activation('relu')(BatchNorm1)
        Conv2D_3_3 = Conv2D(int(num_filter), (3,3), use_bias=False ,padding='same')(relu1)
        if dropout_rate>0:
          Conv2D_3_3 = Dropout(dropout_rate)(Conv2D_3_3)
          
        concat = Concatenate(axis=-1)([temp,Conv2D_3_3])
        temp = concat
        
    return temp

In [0]:
def add_transition(input, num_filter = 32, dropout_rate = 0.2):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    Conv2D_BottleNeck = Conv2D(int(num_filter*compression), (1,1), use_bias=False ,padding='same')(relu)
    if dropout_rate>0:
      Conv2D_BottleNeck = Dropout(dropout_rate)(Conv2D_BottleNeck)
      
    avg = AveragePooling2D(pool_size=(2,2))(Conv2D_BottleNeck)
    
    return avg

In [0]:
def output_layer(input):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    AvgPooling = AveragePooling2D(pool_size=(2,2))(relu)
    flat = Flatten()(AvgPooling)
    output = Dense(num_classes, activation='softmax')(flat)
    
    return output

In [0]:
num_filter = 32
dropout_rate = 0.2
l = 8
#input = Input(shape=(28, 28, 3))
input = Input(shape=(img_height, img_width, channel))
First_Conv2D = Conv2D(num_filter, (3,3), use_bias=False ,padding='same')(input)
#First_In = MaxPooling2D(pool_size=(2,2))(First_Conv2D)

First_Block = add_denseblock(First_Conv2D, num_filter, dropout_rate)
First_Transition = add_transition(First_Block, num_filter, dropout_rate)

Second_Block = add_denseblock(First_Transition, num_filter, dropout_rate)
Second_Transition = add_transition(Second_Block, num_filter, dropout_rate)

Third_Block = add_denseblock(Second_Transition, num_filter, dropout_rate)
Third_Transition = add_transition(Third_Block, num_filter, dropout_rate)

#Fourth_Block = add_denseblock(Third_Transition, num_filter, dropout_rate)
#Fourth_Transition = add_transition(Fourth_Block, num_filter, dropout_rate)

Last_Block = add_denseblock(Third_Transition,  num_filter, dropout_rate)

output = output_layer(Last_Block)

In [0]:
model = Model(inputs=[input], outputs=[output])
model.summary()

In [0]:
# determine Loss function and Optimizer

#LR_update = ReduceLROnPlateau(monitor = "val_acc", factor = 0.01,
#  patience = 40, verbose = 1, mode = "auto",
#  min_delta = 1e-05, cooldown = 0, min_lr = 0.00001)

def step_decay(epochs):
   initial_lrate = 0.1
   drop = 0.1
   epochs_drop = 60.0
   lrate = initial_lrate * math.pow(drop,  
           math.floor((1+epochs)/epochs_drop))
   return lrate

lrate = LearningRateScheduler(step_decay,verbose=1)

model.compile(loss='categorical_crossentropy',
              optimizer=SGD(lr=0.1, momentum=0.9),
              metrics=['accuracy'])

In [0]:
filepath="/home/adityajindal4/.jupyter/Densenet-weights/weights-improvement-{epoch:02d}-{val_acc:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')

In [0]:
model.fit_generator(imgAug.flow(x_train, y_train, batch_size=batch_size),
                    epochs=epochs,steps_per_epoch=2*(x_train.shape[0] / batch_size),
                    verbose=1,
                    validation_data=(x_test, y_test), callbacks = [lrate,checkpoint])

In [0]:
def step_decay(epochs):
   initial_lrate = 0.1
   drop = 0.1
   epochs_drop = 30.0
   lrate_new = initial_lrate * math.pow(drop,  
           math.floor((1+50)/epochs_drop))
   return lrate_new

lrate_new = LearningRateScheduler(step_decay,verbose=1)
model.load_weights("/home/adityajindal4/.jupyter/Densenet-weights/weights-improvement-119-0.89.hdf5")

model.compile(loss='categorical_crossentropy',
              optimizer=SGD(lr=0.1, momentum=0.9,nesterov=True),
              metrics=['accuracy'])

In [0]:
model.fit(x_train, y_train, batch_size=128, epochs=50, 
           verbose=1, validation_data=(x_test, y_test), callbacks = [lrate_new,checkpoint])


In [0]:
model.load_weights("/home/adityajindal4/.jupyter/Densenet-weights/weights-improvement-30-0.91.hdf5")

model.compile(loss='categorical_crossentropy',
              optimizer=SGD(lr=0.01, momentum=0.9,nesterov=True),
              metrics=['accuracy'])

In [0]:
model.fit(x_train, y_train, batch_size=256, epochs=50, 
           verbose=1, validation_data=(x_test, y_test), callbacks = [checkpoint])

In [0]:
# Test the model
score = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])